In [41]:
# Styling notebook
from IPython.core.display import HTML
def css_styling():
    styles = open("./styles/custom.css", "r").read()
    return HTML(styles)
css_styling()

**Abstract Syntax Tree**

In [44]:
from abc import *
class AST(ABC):
    def traverse(self) :
        # print(self.__class__.__name__)
        pass

class Stmt(AST):
    def __init__(self) :
        pass

class CompoundStmt(Stmt) :
    def __init__(self, stmt1, stmt2) :
        self.stmt1 = stmt1                   # First statement
        self.stmt2 = stmt2                   # Remaining ones
        
class Assignment(Stmt) :
    def __init__(self, lvalue, expr) :
        self.lvalue = lvalue                 # LValue
        self.expr   = expr                   # RValue
    
    def traverse(self) :
        super().traverse()
        self.lvalue.traverse()
        self.expr.traverse()
        print("STORE")

        
class AExpr(AST) :
    def __init__(self) :
        pass

class IDNode(AExpr) :
    def __init__(self, name) :
        self.name = name
        
    def traverse(self) :
        super().traverse()
        print(self.name)
        
class ConstNode(AExpr) :
    def __init__(self, value) :
        self.value = value
    
    def traverse(self) :
        super().traverse()
        print(self.value)
    
class OpNode(AExpr) :
    def __init__(self, op, left, right) :
        self.op = op
        self.left = left
        self.right = right 
    
    def traverse(self) :
        super().traverse()
        self.left.traverse()
        self.right.traverse()
        print(self.op)

    

In [45]:
operand1 = IDNode('a')
operand2 = ConstNode(1)
expr = OpNode('+',operand1,operand2)
left = IDNode('a')

s = Assignment(left, expr)
s.traverse()


a
a
1
+
STORE


In [81]:
from Scanner import Scanner

class Parser :
    def __init__(self, source) :
        self.source = source
        self.scanner = Scanner(source)
        self.start = self.Stmt
        self.currentToken = None
        self.currentValue = None
    
    def error(self,message) :
        print("Error: " + message)
       
    def emit(self,tag,item) :
        self.IR.append([tag,item])
        
    def scan(self) :
        self.currentToken, self.currentValue = self.scanner.nextToken()
        
    def expect(self,token) :
        if self.currentToken != token :
            self.error(token + ' expected')
        else :
            self.scan()
    
    def parse(self) :
        self.scanner.init()
        self.scan()
        return self.start()
    
    def Stmt(self) :
        return self.Assign()
    
    def Assign(self) :
        lvalue = self.LVal()
        self.expect('assignSym')
        expr = self.AExpr()
        return Assignment(lvalue,expr)
        
    def LVal(self) :
        if self.currentToken == 'identifier' :
            v = self.currentValue
            self.scan()
            return IDNode(v)

    def AExpr(self) :
        term1 = self.Term()
        if self.currentToken not in {'plusSym', 'minusSym'} :
            return term1
        root = OpNode(None,term1,None)
        current = root
        while self.currentToken in {'plusSym', 'minusSym'} :
            op = self.currentToken
            self.scan()
            current.op = op
            term = self.Term()
            # current.traverse()
            if self.currentToken not in {'plusSym', 'minusSym'} :
                current.right = term
            else :
                current.right = OpNode(None,term,None)
                current = current.right
        return root    

    def Term(self) :
        factor1 = self.Factor()
        while self.currentToken in {'timesSym', 'divSym'} :
            op = self.currentToken
            self.scan()
            self.Factor()
        return factor1
    
    def Factor(self) :
        if self.currentToken == 'lParen' :
            self.scan()
            root = self.AExpr()
            self.expect('rParen')
            return root
                
        if self.currentToken == 'identifier' :
            v = self.currentValue
            self.scan()
            return IDNode(v)
        
        if self.currentToken == 'numConstant' :
            v = self.currentValue
            self.scan()
            return ConstNode(v)


In [82]:
s = "xyz = xyz - 1 + 5"
p = Parser(s)
p.parse().traverse()

xyz
xyz
1
5
plusSym
minusSym
STORE
